In [1]:
!pip install azure-storage-blob
!pip install azure-storage-blob # Microsoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar

In [3]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [4]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

In [5]:
import json
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO

# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Connection string from configuration file
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]

# Name of the container
CONTAINER_AZURE = 'nypdshooting24'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

# Initialize an empty DataFrame to collect data
nypd_df = pd.DataFrame()




In [6]:
from azure.storage.blob import BlobServiceClient
connection_string = "DefaultEndpointsProtocol=https;AccountName=nypdshooting24;AccountKey=mIpjvXnwkckgjWqaDVsQizV2Q4nQSwWxEKEZ36B/zZYAlsVsVJs9QXf5jnNfIM+LkIY08BcGZGf2+AStS0Amhw==;EndpointSuffix=core.windows.net"
container_name = "blob"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)


In [7]:
# Create the container
container_client.create_container()

ResourceExistsError: The specified container already exists.
RequestId:c9bfe361-501e-0068-70e8-a67826000000
Time:2024-05-15T16:53:40.5039721Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:c9bfe361-501e-0068-70e8-a67826000000
Time:2024-05-15T16:53:40.5039721Z</Message></Error>

In [6]:

# Create a BlobServiceClient using the connection string
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get a client to interact with the container
container_client = blob_service_client.get_container_client(container_name)

# List all blobs in the container
blob_list = container_client.list_blobs()

# Process each blob
for blob in blob_list:
    print(f"Processing blob: {blob.name}")

    # Get the blob client
    blob_client = container_client.get_blob_client(blob)

    # Download the blob data
    blob_data = blob_client.download_blob()

    # Read the blob content  (text-based)
    blob_content = blob_data.readall().decode('utf-8')



NameError: name 'connection_string' is not defined

In [7]:
URL = "https://data.cityofnewyork.us/resource/833y-fsy8.csv"

In [8]:
df_raw = pd.read_csv(URL, engine = "pyarrow")

df_raw.head()

,incident_key,occur_date,occur_time,boro,loc_of_occur_desc,precinct,jurisdiction_code,loc_classfctn_desc,location_desc,statistical_murder_flag,...,perp_sex,perp_race,vic_age_group,vic_sex,vic_race,x_coord_cd,y_coord_cd,latitude,longitude,geocoded_column
0,279683077,2023-12-29,03:43:00,QUEENS,INSIDE,113,0,COMMERCIAL,RESTAURANT/DINER,False,...,(null),(null),25-44,M,BLACK,1047268,189119,40.685543,-73.772774,POINT (-73.772774 40.685543)
1,279709792,2023-12-29,21:22:00,BROOKLYN,OUTSIDE,75,0,STREET,(null),False,...,M,BLACK,25-44,M,BLACK,1018514,178646,40.656953,-73.876507,POINT (-73.876507 40.656953)
2,279758069,2023-12-29,18:40:00,BRONX,OUTSIDE,40,0,STREET,(null),False,...,(null),(null),65+,M,BLACK HISPANIC,1010564,235262,40.812379,-73.904940,POINT (-73.90494 40.812379)
3,279609499,2023-12-27,19:47:00,BRONX,OUTSIDE,42,0,STREET,(null),False,...,(null),(null),<18,M,BLACK HISPANIC,1015730,240807,40.827580,-73.886248,POINT (-73.886248 40.82758)
4,279547332,2023-12-26,23:31:00,BRONX,OUTSIDE,46,0,STREET,(null),False,...,M,WHITE HISPANIC,45-64,M,WHITE,1011035,250045,40.852951,-73.903178,POINT (-73.903178 40.852951)


In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   incident_key             1000 non-null   int64         
 1   occur_date               1000 non-null   datetime64[ns]
 2   occur_time               1000 non-null   object        
 3   boro                     1000 non-null   object        
 4   loc_of_occur_desc        1000 non-null   object        
 5   precinct                 1000 non-null   int64         
 6   jurisdiction_code        1000 non-null   int64         
 7   loc_classfctn_desc       1000 non-null   object        
 8   location_desc            1000 non-null   object        
 9   statistical_murder_flag  1000 non-null   bool          
 10  perp_age_group           1000 non-null   object        
 11  perp_sex                 1000 non-null   object        
 12  perp_race                1000 non-n

In [10]:
# Now we will look at the specific datatypes and perform modifications, if needed
df_raw.dtypes
df_raw['occur_date'] = pd.to_datetime(df_raw['occur_date'], errors='coerce')
df_raw['occur_time'] = pd.to_datetime(df_raw['occur_time'], errors='coerce')
df_raw.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   incident_key             1000 non-null   int64         
 1   occur_date               1000 non-null   datetime64[ns]
 2   occur_time               0 non-null      datetime64[ns]
 3   boro                     1000 non-null   object        
 4   loc_of_occur_desc        1000 non-null   object        
 5   precinct                 1000 non-null   int64         
 6   jurisdiction_code        1000 non-null   int64         
 7   loc_classfctn_desc       1000 non-null   object        
 8   location_desc            1000 non-null   object        
 9   statistical_murder_flag  1000 non-null   bool          
 10  perp_age_group           1000 non-null   object        
 11  perp_sex                 1000 non-null   object        
 12  perp_race                1000 non-n

In [11]:

df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   incident_key             1000 non-null   int64         
 1   occur_date               1000 non-null   datetime64[ns]
 2   occur_time               0 non-null      datetime64[ns]
 3   boro                     1000 non-null   object        
 4   loc_of_occur_desc        1000 non-null   object        
 5   precinct                 1000 non-null   int64         
 6   jurisdiction_code        1000 non-null   int64         
 7   loc_classfctn_desc       1000 non-null   object        
 8   location_desc            1000 non-null   object        
 9   statistical_murder_flag  1000 non-null   bool          
 10  perp_age_group           1000 non-null   object        
 11  perp_sex                 1000 non-null   object        
 12  perp_race                1000 non-n

In [12]:
df_raw.columns

Index(['incident_key', 'occur_date', 'occur_time', 'boro', 'loc_of_occur_desc',
       'precinct', 'jurisdiction_code', 'loc_classfctn_desc', 'location_desc',
       'statistical_murder_flag', 'perp_age_group', 'perp_sex', 'perp_race',
       'vic_age_group', 'vic_sex', 'vic_race', 'x_coord_cd', 'y_coord_cd',
       'latitude', 'longitude', 'geocoded_column'],
      dtype='object')

In [13]:
## Creating the dimensions


In [14]:
# Create Date Dimension

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

start_date = pd.to_datetime('2001-08-01')
end_date = pd.to_datetime('2021-12-31')
# Create a DataFrame for the date dimension
dim_date = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})

dim_date.head(25)

# Extract attributes
dim_date['year_number'] = dim_date['date'].dt.year
dim_date['quarter_number'] = dim_date['date'].dt.quarter
dim_date['month_number'] = dim_date['date'].dt.month
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['day_number'] = dim_date['date'].dt.day
dim_date['day_name'] = dim_date['date'].dt.strftime('%A')
dim_date['date_iso_format'] = dim_date['date'].dt.strftime('%Y-%m-%d')
dim_date['date_id'] = dim_date['date'].dt.strftime('%Y%m%d')


dim_date['date_iso_format'] = dim_date['date'].dt.date
# Add week of the month and week of the year
dim_date['week_of_month'] = dim_date['date'].apply(week_of_month)
dim_date['week_of_year'] = dim_date['date'].dt.strftime('%U')

new_order = ['date_id', 'date_iso_format','year_number','month_number','day_number','quarter_number','month_name','day_name','week_of_year','week_of_month']
dim_date = dim_date[new_order]

dim_date.head(25)

,date_id,date_iso_format,year_number,month_number,day_number,quarter_number,month_name,day_name,week_of_year,week_of_month
0,20010801,2001-08-01,2001,8,1,3,August,Wednesday,30,1
1,20010802,2001-08-02,2001,8,2,3,August,Thursday,30,1
2,20010803,2001-08-03,2001,8,3,3,August,Friday,30,1
3,20010804,2001-08-04,2001,8,4,3,August,Saturday,30,1
4,20010805,2001-08-05,2001,8,5,3,August,Sunday,31,1
5,20010806,2001-08-06,2001,8,6,3,August,Monday,31,1
6,20010807,2001-08-07,2001,8,7,3,August,Tuesday,31,1
7,20010808,2001-08-08,2001,8,8,3,August,Wednesday,31,2
8,20010809,2001-08-09,2001,8,9,3,August,Thursday,31,2
9,20010810,2001-08-10,2001,8,10,3,August,Friday,31,2


In [15]:
## Create dim_location

# Assuming REsales_df is your main dataset with 'town' and 'address' columns
# Extract unique pairs of town and address
unique_pairs = df_raw[['boro']].drop_duplicates().reset_index(drop=True)

# Add a unique location_id starting from 1 to these unique pairs
unique_pairs['boro'] = range(1, len(unique_pairs) + 1)

# Create dim_location with the desired column order
dim_location = unique_pairs['boro']

# Display the dim_location DataFrame
print(dim_location)

0    1
1    2
2    3
3    4
4    5
Name: boro, dtype: int64


In [16]:
## Create dim_precinct
# Extract unique property types from the 'precinct' column
unique_precincts = pd.DataFrame(df_raw['precinct'].unique(), columns=['precinct'])

# Add a unique property_type_id starting from 1 to these unique precincts
unique_precincts['precinct_id'] = range(1, len(unique_precincts) + 1)

# Optionally, if 'jurisdiction_code' is also a unique identifier for each precinct, you can include it in the dimension table
unique_precincts['jurisdiction_code'] = df_raw.groupby('precinct')['jurisdiction_code'].first().values

# Create dim_precinct with the desired column order
dim_precinct = unique_precincts[['precinct_id', 'precinct', 'jurisdiction_code']]

# Display the dim_precinct DataFrame
print(dim_precinct.head(10))


   precinct_id  precinct  jurisdiction_code
0            1       113                  0
1            2        75                  0
2            3        40                  2
3            4        42                  0
4            5        46                  0
5            6       106                  0
6            7        73                  0
7            8        67                  0
8            9        69                  2
9           10        26                  0


In [17]:
#Create dim_loc_of_occur_desc
dim_loc_of_occur_desc = pd.DataFrame({'loc_of_occur_desc': df_raw['loc_of_occur_desc'].unique()})
dim_loc_of_occur_desc['loc_of_occur_desc_id'] = range(1, len(dim_loc_of_occur_desc) + 1)
print("\nDimension - loc_of_occur_desc:")
print(dim_loc_of_occur_desc.head())




Dimension - loc_of_occur_desc:
  loc_of_occur_desc  loc_of_occur_desc_id
0            INSIDE                     1
1           OUTSIDE                     2


In [18]:
# Create dim_loc_classfctn_desc
dim_loc_classfctn_desc = pd.DataFrame({'loc_classfctn_desc': df_raw['loc_classfctn_desc'].unique()})
dim_loc_classfctn_desc['loc_classfctn_desc_id'] = range(1, len(dim_loc_classfctn_desc) + 1)
print("\nDimension - loc_classfctn_desc:")
print(dim_loc_classfctn_desc.head())


Dimension - loc_classfctn_desc:
  loc_classfctn_desc  loc_classfctn_desc_id
0         COMMERCIAL                      1
1             STREET                      2
2           DWELLING                      3
3            HOUSING                      4
4             (null)                      5


In [19]:
df_raw = df_raw.rename(columns={
    'incident_key': 'fact_id',
    'boro': 'location_id'
})


In [20]:
# Create fact table

facts_nypd = pd.DataFrame ({
    "incident_key": df_raw ["fact_id"],
    "occur_date": df_raw ["occur_date"],
    "occur_time": df_raw ["occur_time"],
    "precinct_id": dim_precinct["precinct_id"],
    "loc_of_occur_desc_id": dim_loc_of_occur_desc["loc_of_occur_desc_id"],
    "loc_classfctn_desc_id": dim_loc_classfctn_desc["loc_classfctn_desc_id"],
    "latitude": df_raw ["latitude"],
    "longitude": df_raw ["longitude"],
    "date_id": dim_date["date_id"]
})
facts_nypd.head()


,incident_key,occur_date,occur_time,precinct_id,loc_of_occur_desc_id,loc_classfctn_desc_id,latitude,longitude,date_id
0,279683077.0,2023-12-29,NaT,1.0,1.0,1.0,40.685543,-73.772774,20010801
1,279709792.0,2023-12-29,NaT,2.0,2.0,2.0,40.656953,-73.876507,20010802
2,279758069.0,2023-12-29,NaT,3.0,NaN,3.0,40.812379,-73.904940,20010803
3,279609499.0,2023-12-27,NaT,4.0,NaN,4.0,40.827580,-73.886248,20010804
4,279547332.0,2023-12-26,NaT,5.0,NaN,5.0,40.852951,-73.903178,20010805


In [21]:
facts_nypd.dropna()

,incident_key,occur_date,occur_time,precinct_id,loc_of_occur_desc_id,loc_classfctn_desc_id,latitude,longitude,date_id


In [22]:
facts_nypd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7458 entries, 0 to 7457
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   incident_key           1000 non-null   float64       
 1   occur_date             1000 non-null   datetime64[ns]
 2   occur_time             0 non-null      datetime64[ns]
 3   precinct_id            71 non-null     float64       
 4   loc_of_occur_desc_id   2 non-null      float64       
 5   loc_classfctn_desc_id  10 non-null     float64       
 6   latitude               963 non-null    float64       
 7   longitude              963 non-null    float64       
 8   date_id                7458 non-null   object        
dtypes: datetime64[ns](2), float64(6), object(1)
memory usage: 524.5+ KB


In [23]:
dim_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7458 entries, 0 to 7457
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date_id          7458 non-null   object
 1   date_iso_format  7458 non-null   object
 2   year_number      7458 non-null   int32 
 3   month_number     7458 non-null   int32 
 4   day_number       7458 non-null   int32 
 5   quarter_number   7458 non-null   int32 
 6   month_name       7458 non-null   object
 7   day_name         7458 non-null   object
 8   week_of_year     7458 non-null   object
 9   week_of_month    7458 non-null   int64 
dtypes: int32(4), int64(1), object(5)
memory usage: 466.2+ KB


In [24]:
dim_date['date_id'] = dim_date['date_id'].astype("int64")
dim_date['date_iso_format'] = dim_date['date_iso_format'].astype("datetime64[ns]")
dim_date['week_of_year'] = dim_date['week_of_year'].astype("int64")

In [25]:
dim_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7458 entries, 0 to 7457
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date_id          7458 non-null   int64         
 1   date_iso_format  7458 non-null   datetime64[ns]
 2   year_number      7458 non-null   int32         
 3   month_number     7458 non-null   int32         
 4   day_number       7458 non-null   int32         
 5   quarter_number   7458 non-null   int32         
 6   month_name       7458 non-null   object        
 7   day_name         7458 non-null   object        
 8   week_of_year     7458 non-null   int64         
 9   week_of_month    7458 non-null   int64         
dtypes: datetime64[ns](1), int32(4), int64(3), object(2)
memory usage: 466.2+ KB


In [26]:
dim_date.head(10)


,date_id,date_iso_format,year_number,month_number,day_number,quarter_number,month_name,day_name,week_of_year,week_of_month
0,20010801,2001-08-01,2001,8,1,3,August,Wednesday,30,1
1,20010802,2001-08-02,2001,8,2,3,August,Thursday,30,1
2,20010803,2001-08-03,2001,8,3,3,August,Friday,30,1
3,20010804,2001-08-04,2001,8,4,3,August,Saturday,30,1
4,20010805,2001-08-05,2001,8,5,3,August,Sunday,31,1
5,20010806,2001-08-06,2001,8,6,3,August,Monday,31,1
6,20010807,2001-08-07,2001,8,7,3,August,Tuesday,31,1
7,20010808,2001-08-08,2001,8,8,3,August,Wednesday,31,2
8,20010809,2001-08-09,2001,8,9,3,August,Thursday,31,2
9,20010810,2001-08-10,2001,8,10,3,August,Friday,31,2


In [27]:
dim_location.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5 entries, 0 to 4
Series name: boro
Non-Null Count  Dtype
--------------  -----
5 non-null      int64
dtypes: int64(1)
memory usage: 168.0 bytes


In [28]:
dim_location.head(10)

0    1
1    2
2    3
3    4
4    5
Name: boro, dtype: int64

In [29]:
dim_precinct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   precinct_id        71 non-null     int64
 1   precinct           71 non-null     int64
 2   jurisdiction_code  71 non-null     int64
dtypes: int64(3)
memory usage: 1.8 KB


In [30]:
dim_precinct.head(10)

,precinct_id,precinct,jurisdiction_code
0,1,113,0
1,2,75,0
2,3,40,2
3,4,42,0
4,5,46,0
5,6,106,0
6,7,73,0
7,8,67,0
8,9,69,2
9,10,26,0


In [31]:
dim_loc_of_occur_desc.info()
dim_loc_of_occur_desc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   loc_of_occur_desc     2 non-null      object
 1   loc_of_occur_desc_id  2 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 160.0+ bytes


,loc_of_occur_desc,loc_of_occur_desc_id
0,INSIDE,1
1,OUTSIDE,2


In [32]:
dim_loc_classfctn_desc.info()
dim_loc_classfctn_desc.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   loc_classfctn_desc     10 non-null     object
 1   loc_classfctn_desc_id  10 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 288.0+ bytes


,loc_classfctn_desc,loc_classfctn_desc_id
0,COMMERCIAL,1
1,STREET,2
2,DWELLING,3
3,HOUSING,4
4,(null),5
5,TRANSIT,6
6,PARKING LOT,7
7,VEHICLE,8
8,OTHER,9
9,PLAYGROUND,10


In [33]:

facts_nypd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7458 entries, 0 to 7457
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   incident_key           1000 non-null   float64       
 1   occur_date             1000 non-null   datetime64[ns]
 2   occur_time             0 non-null      datetime64[ns]
 3   precinct_id            71 non-null     float64       
 4   loc_of_occur_desc_id   2 non-null      float64       
 5   loc_classfctn_desc_id  10 non-null     float64       
 6   latitude               963 non-null    float64       
 7   longitude              963 non-null    float64       
 8   date_id                7458 non-null   object        
dtypes: datetime64[ns](2), float64(6), object(1)
memory usage: 524.5+ KB


In [34]:
facts_nypd.head(10)

,incident_key,occur_date,occur_time,precinct_id,loc_of_occur_desc_id,loc_classfctn_desc_id,latitude,longitude,date_id
0,279683077.0,2023-12-29,NaT,1.0,1.0,1.0,40.685543,-73.772774,20010801
1,279709792.0,2023-12-29,NaT,2.0,2.0,2.0,40.656953,-73.876507,20010802
2,279758069.0,2023-12-29,NaT,3.0,NaN,3.0,40.812379,-73.904940,20010803
3,279609499.0,2023-12-27,NaT,4.0,NaN,4.0,40.827580,-73.886248,20010804
4,279547332.0,2023-12-26,NaT,5.0,NaN,5.0,40.852951,-73.903178,20010805
5,279547333.0,2023-12-26,NaT,6.0,NaN,6.0,40.688883,-73.817347,20010806
6,279547333.0,2023-12-26,NaT,7.0,NaN,7.0,40.688883,-73.817347,20010807
7,279500313.0,2023-12-26,NaT,8.0,NaN,8.0,40.658150,-73.911701,20010808
8,279547333.0,2023-12-26,NaT,9.0,NaN,9.0,40.688883,-73.817347,20010809
9,279547333.0,2023-12-26,NaT,10.0,NaN,10.0,40.688883,-73.817347,20010810


In [35]:
from sqlalchemy import create_engine

# Define your PostgreSQL connection details
username = 'jennienguyen'
password = 'Khanhha2601!'  # It's recommended to use environment variables for security
hostname = 'jennienguyen.postgres.database.azure.com'
database_name = 'postgres'  # Replace with the name of the database you want to connect to

# Create the database connection URL
database_url = f'postgresql://{username}:{password}@{hostname}/{database_name}'

# Create an SQLAlchemy engine
engine = create_engine(database_url)

# Test the connection
try:
    with engine.connect() as connection:
        print("Connected successfully!")
except Exception as e:
    print(f"Connection failed: {e}")


Connected successfully!


In [36]:
pip install sqlalchemy

In [37]:
dim_location.to_sql('dim_location', con = engine, if_exists = 'append', index=False)

5

In [38]:
dim_date.to_sql('dim_date', con = engine, if_exists = 'append', index=False)

458

In [39]:
dim_precinct.to_sql('dim_precinct', con = engine, if_exists = 'append', index=False)

71

In [40]:
dim_loc_of_occur_desc.to_sql('dim_loc_of_occur_desc', con = engine, if_exists = 'append', index=False)

2

In [41]:
dim_loc_classfctn_desc.to_sql('dim_loc_classfctn_desc', con = engine, if_exists = 'append', index=False)

10

In [42]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming the engine is already created and set up correctly
try:
    df_raw.to_sql('facts_nypd', engine, if_exists='append', index=False)
    print("Data successfully inserted into facts_nypd.")
except Exception as e:
    print("Failed to insert data:", e)

Data successfully inserted into facts_nypd.
